<a href="https://colab.research.google.com/github/brscn2/deeplearning-for-coders-work/blob/main/ch7_state_of_the_art_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.IMAGENETTE)

In [ ]:
# Using presizing with item_tfms
dblock = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
                   get_items = get_image_files,
                   get_y = parent_label,
                   item_tfms = Resize(460),
                   batch_tfms = aug_transforms(size = 224, min_scale = 0.75))

dls = dblock.dataloaders(path, bs=64)

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.619994,2.246135,0.360344,02:17
1,1.216281,1.859482,0.482450,02:20
2,0.919811,1.162771,0.671770,02:21
3,0.719671,0.671101,0.790515,02:21
4,0.579252,0.536128,0.828977,02:20


In [ ]:
# Needs normalization (mean = 0, std = 1)
x, y = dls.one_batch()
x.mean(dim = [0, 2, 3]), x.std(dim = [0, 2, 3])

(TensorImage([0.4871, 0.4696, 0.4480], device='cuda:0'),
 TensorImage([0.2851, 0.2834, 0.3042], device='cuda:0'))

In [ ]:
# Adding Normalize to batch transforms
def get_dls(bs, size):
    dblock = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
                       get_items = get_image_files,
                       get_y = parent_label,
                       item_tfms = Resize(460),
                       batch_tfms = [*aug_transforms(size = size, min_scale = 0.75),
                                     Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs = bs)

dls = get_dls(64, 224)

x, y = dls.one_batch()
x.mean(dim = [0, 2, 3]), x.std(dim = [0, 2, 3])

(TensorImage([-0.2029, -0.0497,  0.1178], device='cuda:0'),
 TensorImage([1.2329, 1.2314, 1.3509], device='cuda:0'))

In [ ]:
model = xresnet50()
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5, 3e-3)

In [ ]:
# Trying Progressive Resizing
## Smaller images most of the epochs to train faster
## Larger images at the end to increase final accuracy

# batch_size = 128, size = 128
dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(), loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.846546,2.548120,0.351008,01:25
1,1.290853,1.653686,0.516430,01:23
2,0.955559,0.916170,0.710232,01:24
3,0.746501,0.695442,0.773338,01:25


In [ ]:
# Switching to larger images
## It is similar to transfer learning, we fine_tune since working with larger images
## is a bit different than smaller images
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.796477,1.175769,0.654220,02:22


epoch,train_loss,valid_loss,accuracy,time
0,0.629447,0.641481,0.797237,02:22
1,0.629303,0.751233,0.777819,02:22
2,0.568875,0.585967,0.821135,02:21
3,0.481560,0.521446,0.842420,02:21
4,0.429044,0.476727,0.858476,02:21


In [ ]:
# Another data augmentation technique: Mixup
## 1) Select another image from the dataset at random
## 2) Select a weight at random
## 3) Take weighted average of the two images using the random weight from 2)
## 4) Take weighted average of the two label encodings using the random weight from 2)
### ! The labels must be in one-hot label encoding.
### Mixup requires more epochs to get a better accuracy

model = xresnet50()
learn = Learner(dls, model = model, loss_func = CrossEntropyLossFlat(),
                metrics = accuracy, cbs = MixUp(1.))
learn.fit_one_cycle(8, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,2.308039,3.918642,0.283794,01:24
1,1.864669,1.994858,0.481703,01:25
2,1.659255,1.142493,0.633682,01:24
3,1.501750,0.961412,0.707244,01:25
4,1.416603,0.812901,0.756161,01:24


In [ ]:
# Label Smoothing
## Encourage the model to be less confident
## more robust to mislabeled data, generalizes better
## Before training adjust labels, replace 1's with a number slightly less than 1
## replace 0's with a number slightly greater than 0

model = xresnet50()
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(),
                metrics = accuracy)
learn.fit_one_cycle(5, 3e-3)